In [59]:
# TO BE REMOVED!!!
%load_ext autoreload
%autoreload 2
import os
import nglview as nv
import ipywidgets
import zipfile
import webbrowser

def show_pdbs(pdbs, surface=False):
    # Load the PDB files
    views = [nv.show_file(pdb) for pdb in pdbs]
    for view in views:
        if surface:
            view.clear()
            view.add_ball_and_stick()
            view.add_surface(color='electrostatic', opacity=0.95)
        view.layout.width = '100%'
    return ipywidgets.HBox(views)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Haddock3 protein protein docking using BioExcel Building Blocks (biobb)
***
This tutorial aims to illustrate the process of **proptein protein docking** using **Haddock3**, step by step, using the **BioExcel Building Blocks library (biobb)**. 
***
**Biobb modules** used:

 - [biobb_haddock](https://github.com/bioexcel/biobb_haddock): Biobb building blocks for the Haddock3 suite.
 
**Auxiliar libraries** used:

 - [nb_conda_kernels](https://github.com/Anaconda-Platform/nb_conda_kernels): Enables a Jupyter Notebook or JupyterLab application in one conda environment to access kernels for Python, R, and other languages found in other environments.
 - [nglview](http://nglviewer.org/#nglview): Jupyter/IPython widget to interactively view molecular structures and trajectories in notebooks.
 - [ipywidgets](https://github.com/jupyter-widgets/ipywidgets): Interactive HTML widgets for Jupyter notebooks and the IPython kernel.

### Conda Installation and Launch

```console
git clone https://github.com/bioexcel/biobb_wf_ligand_parameterization.git
cd biobb_wf_ligand_parameterization
conda env create -f conda_env/environment.yml
conda activate biobb_ligand_parameterization_tutorial
jupyter-nbextension enable --py --user widgetsnbextension
jupyter-nbextension enable --py --user nglview
jupyter-notebook biobb_wf_ligand_parameterization/notebooks/biobb_ligand_parameterization_tutorial.ipynb
  ``` 

***
### Pipeline steps:
 1. [Input Parameters](#input)
 2. [Create topology](#fetch)
 3. [CAPRI evaluation](#addh)
 4. [Select Top structures](#min)
 5. [Flexible Refinement](#acpype)
 6. [2nd CAPRI evalutation](#output)
 7. [Energy Minimization Refinement](#questions)
 8. [Energy Minimization Refinement](#questions)
 
***
![](https://bioexcel.eu/wp-content/uploads/2019/04/Bioexcell_logo_1080px_transp.png)
***

<a id="input"></a>
***
## Input parameters
**Input parameters** needed:
 - **ligandCode**: 3-letter code of the ligand structure (e.g. IBP)
 - **mol_charge**: Molecule net charge (e.g. -1)
 - **pH**: Acidity or alkalinity for the small molecule. Hydrogen atoms will be added according to this pH. (e.g. 7.4)

# HADDOCK3 PPI tutorial

In [ ]:
# HADDOCK 3: https://www.bonvinlab.org/haddock3/examples.html#docking-protein-protein
# HADDOCK 2.4 (more info): https://www.bonvinlab.org/education/HADDOCK24/HADDOCK24-protein-protein-basic/
data_pth = '/home/rchaves/repo/biobb_haddock/biobb_haddock/test/data/haddock/'
mol1_input_pdb_path = data_pth + 'e2aP_1F3G.pdb'
mol2_input_pdb_path = data_pth + 'hpr_ensemble.pdb'
restraints_table_path = data_pth + 'e2a-hpr_air.tbl'
reference_pdb_path = data_pth + 'e2a-hpr_1GGR.pdb'
out_path = '.data/ppi-tuto'
binary_path = 'haddock3'

## Create topology

In [48]:
from biobb_haddock.haddock.topology import topology

step_idx = 0
mol1_output_top_zip_path = f'{out_path}/{step_idx}/top_mol1.zip'
mol2_output_top_zip_path = f'{out_path}/{step_idx}/top_mol2.zip'
wf_topology              = f'{out_path}/{step_idx}/wf_topology.zip'

topology(mol1_input_pdb_path        = mol1_input_pdb_path,
         mol2_input_pdb_path        = mol2_input_pdb_path,
         mol1_output_top_zip_path   = mol1_output_top_zip_path,
         mol2_output_top_zip_path   = mol2_output_top_zip_path,
         output_haddock_wf_data_zip = wf_topology)

os.remove('log.out'), os.remove('log.err')

2025-04-03 12:45:23,007 [MainThread  ] [INFO ]  Module: biobb_haddock.haddock.topology Version: 5.0.0
2025-04-03 12:45:23,007 [MainThread  ] [INFO ]  /home/rchaves/repo/ab_design/biobb_wf/sandbox_3e30d09e-00ee-472b-87cb-c1325a528a55 directory successfully created
2025-04-03 12:45:23,008 [MainThread  ] [INFO ]  Copy: /home/rchaves/repo/biobb_haddock/biobb_haddock/test/data/haddock/e2aP_1F3G.pdb to /home/rchaves/repo/ab_design/biobb_wf/sandbox_3e30d09e-00ee-472b-87cb-c1325a528a55
2025-04-03 12:45:23,009 [MainThread  ] [INFO ]  Copy: /home/rchaves/repo/biobb_haddock/biobb_haddock/test/data/haddock/hpr_ensemble.pdb to /home/rchaves/repo/ab_design/biobb_wf/sandbox_3e30d09e-00ee-472b-87cb-c1325a528a55
2025-04-03 12:45:23,009 [MainThread  ] [INFO ]  haddock3 0488d222-72a5-49b3-924e-ca49e00d1d0a/haddock.cfg



2025-04-03 12:45:25,485 [MainThread  ] [INFO ]  Executing: haddock3 0488d222-72a5-49b3-924e-ca49e00d1d0a/haddock.cfg...
2025-04-03 12:45:25,486 [MainThread  ] [INFO ]  Exit code: 0
2025-04-03 12:45:25,487 [MainThread  ] [INFO ]  [2025-04-03 12:45:23,578 cli INFO] 
##############################################
#                                            #
#                 HADDOCK3                   #
#                                            #
##############################################

!! Some of the HADDOCK3 components use CNS (Crystallographic and NMR System) which is free of use for non-profit applications. !!
!! For commercial use it is your own responsibility to have a proper license. !!
!! For details refer to the DISCLAIMER file in the HADDOCK3 repository. !!

Starting HADDOCK3 v2024.10.0b7 on 2025-04-03 12:45:00

Python 3.9.21 | packaged by conda-forge | (main, Dec  5 2024, 13:51:40) 
[GCC 13.3.0]

[2025-04-03 12:45:23,682 libworkflow INFO] Reading instructions step 0

(None, None)

## Rigid Body sampling

In [49]:
from biobb_haddock.haddock.rigid_body import rigid_body

properties={
    'cfg': {
        'tolerance': 5,
        'sampling': 20, # 1000
    },
    'remove_tmp' : True,
}

step_idx = 1
docking_output_zip_path = f'{out_path}/{step_idx}/docking.zip'
wf_rigidbody            = f'{out_path}/{step_idx}/wf_rigidbody.zip'

rigid_body(input_haddock_wf_data_zip     = wf_topology,
           docking_output_zip_path       = docking_output_zip_path,
           ambig_restraints_table_path   = restraints_table_path,
           unambig_restraints_table_path = restraints_table_path,
           hb_restraints_table_path      = restraints_table_path,
           output_haddock_wf_data_zip    = wf_rigidbody,
           properties                    = properties)

os.remove('log.out'), os.remove('log.err')

2025-04-03 12:45:25,558 [MainThread  ] [INFO ]  Module: biobb_haddock.haddock.rigid_body Version: 5.0.0


2025-04-03 12:45:25,559 [MainThread  ] [INFO ]  /home/rchaves/repo/ab_design/biobb_wf/sandbox_5e2c650a-c673-46ba-85ab-4f682c94f97a directory successfully created
2025-04-03 12:45:25,559 [MainThread  ] [INFO ]  Copy: /home/rchaves/repo/biobb_haddock/biobb_haddock/test/data/haddock/e2a-hpr_air.tbl to /home/rchaves/repo/ab_design/biobb_wf/sandbox_5e2c650a-c673-46ba-85ab-4f682c94f97a
2025-04-03 12:45:25,560 [MainThread  ] [INFO ]  Copy: /home/rchaves/repo/biobb_haddock/biobb_haddock/test/data/haddock/e2a-hpr_air.tbl to /home/rchaves/repo/ab_design/biobb_wf/sandbox_5e2c650a-c673-46ba-85ab-4f682c94f97a
2025-04-03 12:45:25,560 [MainThread  ] [INFO ]  Copy: /home/rchaves/repo/biobb_haddock/biobb_haddock/test/data/haddock/e2a-hpr_air.tbl to /home/rchaves/repo/ab_design/biobb_wf/sandbox_5e2c650a-c673-46ba-85ab-4f682c94f97a
2025-04-03 12:45:25,571 [MainThread  ] [INFO ]  Extracting: /home/rchaves/repo/ab_design/biobb_wf/ppi-tuto/0/wf_topology.zip
2025-04-03 12:45:25,572 [MainThread  ] [INFO ]  to

(None, None)

## CAPRI evaluation

In [50]:
from biobb_haddock.haddock.capri_eval import capri_eval

step_idx = 2
output_evaluation_zip_path = f'{out_path}/{step_idx}/caprieval.zip'
wf_caprieval               = f'{out_path}/{step_idx}/wf_caprieval.zip'

capri_eval(input_haddock_wf_data_zip  = wf_rigidbody,
           reference_pdb_path         = reference_pdb_path,
           output_evaluation_zip_path = output_evaluation_zip_path,
           output_haddock_wf_data_zip = wf_caprieval,
         properties = {'remove_tmp': True})

os.remove('log.out'), os.remove('log.err')

2025-04-03 12:46:01,347 [MainThread  ] [INFO ]  Module: biobb_haddock.haddock.capri_eval Version: 5.0.0
2025-04-03 12:46:01,348 [MainThread  ] [INFO ]  /home/rchaves/repo/ab_design/biobb_wf/sandbox_daf1b120-aa57-465a-92d0-9ccb9b04fa87 directory successfully created
2025-04-03 12:46:01,349 [MainThread  ] [INFO ]  Copy: /home/rchaves/repo/biobb_haddock/biobb_haddock/test/data/haddock/e2a-hpr_1GGR.pdb to /home/rchaves/repo/ab_design/biobb_wf/sandbox_daf1b120-aa57-465a-92d0-9ccb9b04fa87
2025-04-03 12:46:01,371 [MainThread  ] [INFO ]  Extracting: /home/rchaves/repo/ab_design/biobb_wf/ppi-tuto/1/wf_rigidbody.zip
2025-04-03 12:46:01,372 [MainThread  ] [INFO ]  to:
2025-04-03 12:46:01,372 [MainThread  ] [INFO ]  ['7783a0b5-59c1-4289-9144-6c52a92843c0/0_topoaa', '7783a0b5-59c1-4289-9144-6c52a92843c0/1_rigidbody', '7783a0b5-59c1-4289-9144-6c52a92843c0/analysis', '7783a0b5-59c1-4289-9144-6c52a92843c0/data', '7783a0b5-59c1-4289-9144-6c52a92843c0/traceback', '7783a0b5-59c1-4289-9144-6c52a92843c0/lo

(None, None)

## Select Top structures

In [52]:
from biobb_haddock.haddock.sele_top import sele_top

properties={
    'cfg': {
        'select': 5,
    },
    'remove_tmp' : True,
}

step_idx = 3
output_selection_zip_path = f'{out_path}/{step_idx}/selected.zip'
wf_seletop                = f'{out_path}/{step_idx}/wf_seletop.zip'

sele_top(input_haddock_wf_data_zip  = wf_caprieval,
         output_selection_zip_path  = output_selection_zip_path,
         output_haddock_wf_data_zip = wf_seletop,
         properties                 = properties)

os.remove('log.out'), os.remove('log.err')

2025-04-03 12:46:53,021 [MainThread  ] [INFO ]  Module: biobb_haddock.haddock.sele_top Version: 5.0.0
2025-04-03 12:46:53,021 [MainThread  ] [INFO ]  /home/rchaves/repo/ab_design/biobb_wf/sandbox_eaa62515-332c-44d9-a101-b06b8a9e99ae directory successfully created
2025-04-03 12:46:53,051 [MainThread  ] [INFO ]  Extracting: /home/rchaves/repo/ab_design/biobb_wf/ppi-tuto/2/wf_caprieval.zip
2025-04-03 12:46:53,051 [MainThread  ] [INFO ]  to:
2025-04-03 12:46:53,051 [MainThread  ] [INFO ]  ['3a7017ce-da60-465c-9e84-8d2d9f6ac8df/0_topoaa', '3a7017ce-da60-465c-9e84-8d2d9f6ac8df/1_rigidbody', '3a7017ce-da60-465c-9e84-8d2d9f6ac8df/2_caprieval', '3a7017ce-da60-465c-9e84-8d2d9f6ac8df/analysis', '3a7017ce-da60-465c-9e84-8d2d9f6ac8df/data', '3a7017ce-da60-465c-9e84-8d2d9f6ac8df/traceback', '3a7017ce-da60-465c-9e84-8d2d9f6ac8df/log', '3a7017ce-da60-465c-9e84-8d2d9f6ac8df/traceback/consensus.tsv', '3a7017ce-da60-465c-9e84-8d2d9f6ac8df/traceback/traceback.tsv', '3a7017ce-da60-465c-9e84-8d2d9f6ac8df/tr

(None, None)

## 2nd CAPRI evalutation

In [18]:
from biobb_haddock.haddock.capri_eval import capri_eval

step_idx = 4
output_evaluation_zip_path2 = f'{out_path}/{step_idx}/caprieval2.zip'
wf_caprieval2               = f'{out_path}/{step_idx}/wf_caprieval2.zip'

capri_eval(input_haddock_wf_data_zip  = wf_seletop,
           reference_pdb_path         = reference_pdb_path,
           output_evaluation_zip_path = output_evaluation_zip_path2,
           output_haddock_wf_data_zip = wf_caprieval2)
 
os.remove('log.out'), os.remove('log.err')

2025-04-03 10:22:02,026 [MainThread  ] [INFO ]  Module: biobb_haddock.haddock.capri_eval Version: 5.0.0
2025-04-03 10:22:02,026 [MainThread  ] [INFO ]  /home/rchaves/repo/ab_design/biobb_wf/sandbox_c23c3e37-8220-40ca-af4a-e492225ab287 directory successfully created
2025-04-03 10:22:02,027 [MainThread  ] [INFO ]  Copy: /home/rchaves/repo/biobb_haddock/biobb_haddock/test/data/haddock/e2a-hpr_1GGR.pdb to /home/rchaves/repo/ab_design/biobb_wf/sandbox_c23c3e37-8220-40ca-af4a-e492225ab287
2025-04-03 10:22:02,065 [MainThread  ] [INFO ]  Extracting: /home/rchaves/repo/ab_design/biobb_wf/ppi-tuto/3/wf_seletop.zip
2025-04-03 10:22:02,065 [MainThread  ] [INFO ]  to:
2025-04-03 10:22:02,066 [MainThread  ] [INFO ]  ['6c0dbfdc-9d53-46df-91be-d3241ffb2c49/0_topoaa', '6c0dbfdc-9d53-46df-91be-d3241ffb2c49/1_rigidbody', '6c0dbfdc-9d53-46df-91be-d3241ffb2c49/2_caprieval', '6c0dbfdc-9d53-46df-91be-d3241ffb2c49/3_seletop', '6c0dbfdc-9d53-46df-91be-d3241ffb2c49/analysis', '6c0dbfdc-9d53-46df-91be-d3241ffb2c

(None, None)

## Flexible Refinement

In [19]:
from biobb_haddock.haddock.flex_ref import flex_ref

step_idx = 5
refinement_output_zip_path = f'{out_path}/{step_idx}/flexref.zip'
wf_flexref                 = f'{out_path}/{step_idx}/wf_flexref.zip'

flex_ref(input_haddock_wf_data_zip  = wf_caprieval2,
         refinement_output_zip_path = refinement_output_zip_path,
         restraints_table_path      = restraints_table_path,
         output_haddock_wf_data_zip = wf_flexref)

os.remove('log.out'), os.remove('log.err')

2025-04-03 10:22:05,589 [MainThread  ] [INFO ]  Module: biobb_haddock.haddock.flex_ref Version: 5.0.0
2025-04-03 10:22:05,589 [MainThread  ] [INFO ]  /home/rchaves/repo/ab_design/biobb_wf/sandbox_574b72c9-2a74-4b5d-8721-6adb30b75501 directory successfully created
2025-04-03 10:22:05,590 [MainThread  ] [INFO ]  Copy: /home/rchaves/repo/biobb_haddock/biobb_haddock/test/data/haddock/e2a-hpr_air.tbl to /home/rchaves/repo/ab_design/biobb_wf/sandbox_574b72c9-2a74-4b5d-8721-6adb30b75501


2025-04-03 10:22:05,642 [MainThread  ] [INFO ]  Extracting: /home/rchaves/repo/ab_design/biobb_wf/ppi-tuto/4/wf_caprieval2.zip
2025-04-03 10:22:05,642 [MainThread  ] [INFO ]  to:
2025-04-03 10:22:05,643 [MainThread  ] [INFO ]  ['da1e47fe-3598-402c-af4f-16607d5d603f/0_topoaa', 'da1e47fe-3598-402c-af4f-16607d5d603f/1_rigidbody', 'da1e47fe-3598-402c-af4f-16607d5d603f/2_caprieval', 'da1e47fe-3598-402c-af4f-16607d5d603f/3_seletop', 'da1e47fe-3598-402c-af4f-16607d5d603f/4_caprieval', 'da1e47fe-3598-402c-af4f-16607d5d603f/analysis', 'da1e47fe-3598-402c-af4f-16607d5d603f/data', 'da1e47fe-3598-402c-af4f-16607d5d603f/traceback', 'da1e47fe-3598-402c-af4f-16607d5d603f/log', 'da1e47fe-3598-402c-af4f-16607d5d603f/traceback/consensus.tsv', 'da1e47fe-3598-402c-af4f-16607d5d603f/traceback/traceback.tsv', 'da1e47fe-3598-402c-af4f-16607d5d603f/traceback/traceback.html', 'da1e47fe-3598-402c-af4f-16607d5d603f/4_caprieval/io.json', 'da1e47fe-3598-402c-af4f-16607d5d603f/4_caprieval/capri_clt.tsv', 'da1e47fe-

(None, None)

## 3nd CAPRI evalutation

In [20]:
from biobb_haddock.haddock.capri_eval import capri_eval

step_idx = 6
output_evaluation_zip_path3 = f'{out_path}/{step_idx}/caprieval3.zip'
wf_caprieval3               = f'{out_path}/{step_idx}/wf_caprieval3.zip'

capri_eval(input_haddock_wf_data_zip  = wf_flexref,
           reference_pdb_path         = reference_pdb_path,
           output_evaluation_zip_path = output_evaluation_zip_path3,
           output_haddock_wf_data_zip = wf_caprieval3)
 
os.remove('log.out'), os.remove('log.err')

2025-04-03 10:26:46,047 [MainThread  ] [INFO ]  Module: biobb_haddock.haddock.capri_eval Version: 5.0.0
2025-04-03 10:26:46,048 [MainThread  ] [INFO ]  /home/rchaves/repo/ab_design/biobb_wf/sandbox_42d56535-aebc-4d56-b940-141f92c93367 directory successfully created
2025-04-03 10:26:46,049 [MainThread  ] [INFO ]  Copy: /home/rchaves/repo/biobb_haddock/biobb_haddock/test/data/haddock/e2a-hpr_1GGR.pdb to /home/rchaves/repo/ab_design/biobb_wf/sandbox_42d56535-aebc-4d56-b940-141f92c93367
2025-04-03 10:26:46,119 [MainThread  ] [INFO ]  Extracting: /home/rchaves/repo/ab_design/biobb_wf/ppi-tuto/5/wf_flexref.zip
2025-04-03 10:26:46,119 [MainThread  ] [INFO ]  to:
2025-04-03 10:26:46,120 [MainThread  ] [INFO ]  ['669efc8f-ea25-4846-a184-f81b37eb103b/0_topoaa', '669efc8f-ea25-4846-a184-f81b37eb103b/1_rigidbody', '669efc8f-ea25-4846-a184-f81b37eb103b/2_caprieval', '669efc8f-ea25-4846-a184-f81b37eb103b/3_seletop', '669efc8f-ea25-4846-a184-f81b37eb103b/4_caprieval', '669efc8f-ea25-4846-a184-f81b37e

(None, None)

## Energy Minimization Refinement

In [21]:
from biobb_haddock.haddock.em_ref import em_ref

step_idx = 7
refinement_output_zip_path = f'{out_path}/{step_idx}/emref.zip'
wf_emref                   = f'{out_path}/{step_idx}/wf_emref.zip'

em_ref(input_haddock_wf_data_zip  = wf_caprieval3,
       refinement_output_zip_path = refinement_output_zip_path,
       restraints_table_path      = restraints_table_path,
       output_haddock_wf_data_zip = wf_emref)

os.remove('log.out'), os.remove('log.err')

2025-04-03 10:26:56,952 [MainThread  ] [INFO ]  Module: biobb_haddock.haddock.em_ref Version: 5.0.0
2025-04-03 10:26:56,952 [MainThread  ] [INFO ]  /home/rchaves/repo/ab_design/biobb_wf/sandbox_6ffae4e0-5277-413e-9717-9d2fb5f11bf4 directory successfully created
2025-04-03 10:26:56,953 [MainThread  ] [INFO ]  Copy: /home/rchaves/repo/biobb_haddock/biobb_haddock/test/data/haddock/e2a-hpr_air.tbl to /home/rchaves/repo/ab_design/biobb_wf/sandbox_6ffae4e0-5277-413e-9717-9d2fb5f11bf4


2025-04-03 10:26:57,052 [MainThread  ] [INFO ]  Extracting: /home/rchaves/repo/ab_design/biobb_wf/ppi-tuto/6/wf_caprieval3.zip
2025-04-03 10:26:57,052 [MainThread  ] [INFO ]  to:
2025-04-03 10:26:57,053 [MainThread  ] [INFO ]  ['2e3ba6b8-e69e-4c42-adb8-a82ce7c33c82/0_topoaa', '2e3ba6b8-e69e-4c42-adb8-a82ce7c33c82/1_rigidbody', '2e3ba6b8-e69e-4c42-adb8-a82ce7c33c82/2_caprieval', '2e3ba6b8-e69e-4c42-adb8-a82ce7c33c82/3_seletop', '2e3ba6b8-e69e-4c42-adb8-a82ce7c33c82/4_caprieval', '2e3ba6b8-e69e-4c42-adb8-a82ce7c33c82/5_flexref', '2e3ba6b8-e69e-4c42-adb8-a82ce7c33c82/6_caprieval', '2e3ba6b8-e69e-4c42-adb8-a82ce7c33c82/analysis', '2e3ba6b8-e69e-4c42-adb8-a82ce7c33c82/data', '2e3ba6b8-e69e-4c42-adb8-a82ce7c33c82/traceback', '2e3ba6b8-e69e-4c42-adb8-a82ce7c33c82/log', '2e3ba6b8-e69e-4c42-adb8-a82ce7c33c82/traceback/consensus.tsv', '2e3ba6b8-e69e-4c42-adb8-a82ce7c33c82/traceback/traceback.tsv', '2e3ba6b8-e69e-4c42-adb8-a82ce7c33c82/traceback/traceback.html', '2e3ba6b8-e69e-4c42-adb8-a82ce7c33

(None, None)

## 4rd CAPRI evaluation

In [22]:
from biobb_haddock.haddock.capri_eval import capri_eval

step_idx = 8
output_evaluation_zip_path4 = f'{out_path}/{step_idx}/caprieval4.zip'
wf_caprieval4               = f'{out_path}/{step_idx}/wf_caprieval4.zip'

capri_eval(input_haddock_wf_data_zip  = wf_emref,
           reference_pdb_path         = reference_pdb_path,
           output_evaluation_zip_path = output_evaluation_zip_path4,
           output_haddock_wf_data_zip = wf_caprieval4)

os.remove('log.out'), os.remove('log.err')

2025-04-03 10:27:17,556 [MainThread  ] [INFO ]  Module: biobb_haddock.haddock.capri_eval Version: 5.0.0
2025-04-03 10:27:17,557 [MainThread  ] [INFO ]  /home/rchaves/repo/ab_design/biobb_wf/sandbox_b6d4bfcf-5e41-4247-bb8f-2c2d3031541e directory successfully created
2025-04-03 10:27:17,558 [MainThread  ] [INFO ]  Copy: /home/rchaves/repo/biobb_haddock/biobb_haddock/test/data/haddock/e2a-hpr_1GGR.pdb to /home/rchaves/repo/ab_design/biobb_wf/sandbox_b6d4bfcf-5e41-4247-bb8f-2c2d3031541e


2025-04-03 10:27:17,664 [MainThread  ] [INFO ]  Extracting: /home/rchaves/repo/ab_design/biobb_wf/ppi-tuto/7/wf_emref.zip
2025-04-03 10:27:17,664 [MainThread  ] [INFO ]  to:
2025-04-03 10:27:17,665 [MainThread  ] [INFO ]  ['f17b2bef-0bba-40b0-9fc7-c790426dcc05/0_topoaa', 'f17b2bef-0bba-40b0-9fc7-c790426dcc05/1_rigidbody', 'f17b2bef-0bba-40b0-9fc7-c790426dcc05/2_caprieval', 'f17b2bef-0bba-40b0-9fc7-c790426dcc05/3_seletop', 'f17b2bef-0bba-40b0-9fc7-c790426dcc05/4_caprieval', 'f17b2bef-0bba-40b0-9fc7-c790426dcc05/5_flexref', 'f17b2bef-0bba-40b0-9fc7-c790426dcc05/6_caprieval', 'f17b2bef-0bba-40b0-9fc7-c790426dcc05/7_emref', 'f17b2bef-0bba-40b0-9fc7-c790426dcc05/analysis', 'f17b2bef-0bba-40b0-9fc7-c790426dcc05/data', 'f17b2bef-0bba-40b0-9fc7-c790426dcc05/traceback', 'f17b2bef-0bba-40b0-9fc7-c790426dcc05/log', 'f17b2bef-0bba-40b0-9fc7-c790426dcc05/traceback/consensus.tsv', 'f17b2bef-0bba-40b0-9fc7-c790426dcc05/traceback/traceback.tsv', 'f17b2bef-0bba-40b0-9fc7-c790426dcc05/traceback/tracebac

(None, None)

## Clustering using FCC

In [ ]:
from biobb_haddock.haddock.clust_fcc import clust_fcc

step_idx = 9
output_cluster_zip_path = f'{out_path}/{step_idx}/clustfcc.zip'
wf_clustfcc             = f'{out_path}/{step_idx}/wf_clustfcc.zip'

clust_fcc(input_haddock_wf_data_zip = wf_caprieval4,
         output_cluster_zip_path    = output_cluster_zip_path,
         output_haddock_wf_data_zip = wf_clustfcc,
         properties                 = {'remove_tmp' : True})

os.remove('log.out'), os.remove('log.err')

2025-04-03 10:36:06,497 [MainThread  ] [INFO ]  Module: biobb_haddock.haddock.clust_fcc Version: 5.0.0
2025-04-03 10:36:06,498 [MainThread  ] [INFO ]  /home/rchaves/repo/ab_design/biobb_wf/sandbox_f9d8962a-b1d5-437b-8208-5a882d903b60 directory successfully created


2025-04-03 10:36:06,604 [MainThread  ] [INFO ]  Extracting: /home/rchaves/repo/ab_design/biobb_wf/ppi-tuto/8/wf_caprieval4.zip
2025-04-03 10:36:06,604 [MainThread  ] [INFO ]  to:
2025-04-03 10:36:06,605 [MainThread  ] [INFO ]  ['522c6eef-d09d-4599-83fc-95af9f2151ad/0_topoaa', '522c6eef-d09d-4599-83fc-95af9f2151ad/1_rigidbody', '522c6eef-d09d-4599-83fc-95af9f2151ad/2_caprieval', '522c6eef-d09d-4599-83fc-95af9f2151ad/3_seletop', '522c6eef-d09d-4599-83fc-95af9f2151ad/4_caprieval', '522c6eef-d09d-4599-83fc-95af9f2151ad/5_flexref', '522c6eef-d09d-4599-83fc-95af9f2151ad/6_caprieval', '522c6eef-d09d-4599-83fc-95af9f2151ad/7_emref', '522c6eef-d09d-4599-83fc-95af9f2151ad/8_caprieval', '522c6eef-d09d-4599-83fc-95af9f2151ad/analysis', '522c6eef-d09d-4599-83fc-95af9f2151ad/data', '522c6eef-d09d-4599-83fc-95af9f2151ad/traceback', '522c6eef-d09d-4599-83fc-95af9f2151ad/log', '522c6eef-d09d-4599-83fc-95af9f2151ad/traceback/consensus.tsv', '522c6eef-d09d-4599-83fc-95af9f2151ad/traceback/traceback.tsv',

(None, None)

## Selecting top clusters

In [26]:
from biobb_haddock.haddock.sele_top_clusts import sele_top_clusts

properties={
    'cfg': {
        'top_models': 4,
    },
    'remove_tmp' : True,
}

step_idx = 10
output_seletopclusts_zip_path = f'{out_path}/{step_idx}/seletopclusts.zip'
wf_seletopclusts              = f'{out_path}/{step_idx}/wf_seletopclusts.zip'

sele_top_clusts(input_haddock_wf_data_zip = wf_clustfcc,
         output_selection_zip_path        = output_seletopclusts_zip_path,
         output_haddock_wf_data_zip       = wf_seletopclusts,
         properties                       = properties)

os.remove('log.out'), os.remove('log.err')

2025-04-03 10:47:41,756 [MainThread  ] [INFO ]  Module: biobb_haddock.haddock.sele_top_clusts Version: 5.0.0
2025-04-03 10:47:41,756 [MainThread  ] [INFO ]  /home/rchaves/repo/ab_design/biobb_wf/sandbox_3621f43c-00da-495f-8215-76b9930f1e1f directory successfully created
2025-04-03 10:47:41,868 [MainThread  ] [INFO ]  Extracting: /home/rchaves/repo/ab_design/biobb_wf/ppi-tuto/9/wf_clustfcc.zip
2025-04-03 10:47:41,869 [MainThread  ] [INFO ]  to:
2025-04-03 10:47:41,870 [MainThread  ] [INFO ]  ['fe24b2b3-e554-42ff-afc8-b459924dee82/0_topoaa', 'fe24b2b3-e554-42ff-afc8-b459924dee82/1_rigidbody', 'fe24b2b3-e554-42ff-afc8-b459924dee82/2_caprieval', 'fe24b2b3-e554-42ff-afc8-b459924dee82/3_seletop', 'fe24b2b3-e554-42ff-afc8-b459924dee82/4_caprieval', 'fe24b2b3-e554-42ff-afc8-b459924dee82/5_flexref', 'fe24b2b3-e554-42ff-afc8-b459924dee82/6_caprieval', 'fe24b2b3-e554-42ff-afc8-b459924dee82/7_emref', 'fe24b2b3-e554-42ff-afc8-b459924dee82/8_caprieval', 'fe24b2b3-e554-42ff-afc8-b459924dee82/9_clustf

(None, None)

## 5th CAPRI evaluation

In [27]:
from biobb_haddock.haddock.capri_eval import capri_eval

step_idx = 11
output_evaluation_zip_path5 = f'{out_path}/{step_idx}/caprieval5.zip'
wf_caprieval5 = f'{out_path}/{step_idx}/wf_caprieval5.zip'

capri_eval(input_haddock_wf_data_zip = wf_seletopclusts,
           reference_pdb_path = reference_pdb_path,
           output_evaluation_zip_path = output_evaluation_zip_path5,
           output_haddock_wf_data_zip = wf_caprieval5)

os.remove('log.out'), os.remove('log.err')

2025-04-03 10:49:26,527 [MainThread  ] [INFO ]  Module: biobb_haddock.haddock.capri_eval Version: 5.0.0
2025-04-03 10:49:26,528 [MainThread  ] [INFO ]  /home/rchaves/repo/ab_design/biobb_wf/sandbox_dfd85170-5929-463d-b01b-2ca8c7f51d1c directory successfully created
2025-04-03 10:49:26,529 [MainThread  ] [INFO ]  Copy: /home/rchaves/repo/biobb_haddock/biobb_haddock/test/data/haddock/e2a-hpr_1GGR.pdb to /home/rchaves/repo/ab_design/biobb_wf/sandbox_dfd85170-5929-463d-b01b-2ca8c7f51d1c


2025-04-03 10:49:26,667 [MainThread  ] [INFO ]  Extracting: /home/rchaves/repo/ab_design/biobb_wf/ppi-tuto/10/wf_seletopclusts.zip
2025-04-03 10:49:26,668 [MainThread  ] [INFO ]  to:
2025-04-03 10:49:26,668 [MainThread  ] [INFO ]  ['4ba7407c-2da5-4fc7-ba6f-9b176b44484d/00_topoaa', '4ba7407c-2da5-4fc7-ba6f-9b176b44484d/01_rigidbody', '4ba7407c-2da5-4fc7-ba6f-9b176b44484d/02_caprieval', '4ba7407c-2da5-4fc7-ba6f-9b176b44484d/03_seletop', '4ba7407c-2da5-4fc7-ba6f-9b176b44484d/04_caprieval', '4ba7407c-2da5-4fc7-ba6f-9b176b44484d/05_flexref', '4ba7407c-2da5-4fc7-ba6f-9b176b44484d/06_caprieval', '4ba7407c-2da5-4fc7-ba6f-9b176b44484d/07_emref', '4ba7407c-2da5-4fc7-ba6f-9b176b44484d/08_caprieval', '4ba7407c-2da5-4fc7-ba6f-9b176b44484d/09_clustfcc', '4ba7407c-2da5-4fc7-ba6f-9b176b44484d/10_seletopclusts', '4ba7407c-2da5-4fc7-ba6f-9b176b44484d/analysis', '4ba7407c-2da5-4fc7-ba6f-9b176b44484d/data', '4ba7407c-2da5-4fc7-ba6f-9b176b44484d/traceback', '4ba7407c-2da5-4fc7-ba6f-9b176b44484d/log', '4ba7

(None, None)

## Results

In [5]:
step_idx = 11

In [6]:
output_evaluation_zip_path5 = f'{out_path}/{step_idx}/caprieval5.zip'
wf_caprieval5 = f'{out_path}/{step_idx}/wf_caprieval5.zip'

In [7]:
import zipfile

# Unzip wf_caprieval5
with zipfile.ZipFile(wf_caprieval5, 'r') as zip_ref:
    zip_ref.extractall(out_path+'/final_results')

In [8]:
import os
import subprocess

# Change directory to the final results folder
os.chdir(out_path + '/final_results')

In [13]:
# Start the HTTP server and store the process
http_server_process = subprocess.Popen(['python3', '-m', 'http.server', '8000'])

# Function to stop the server when needed
def stop_http_server():
    http_server_process.terminate()
    http_server_process.wait()
    print("HTTP server stopped.")

NameError: name 'subprocess' is not defined

In [11]:
stop_http_server()

HTTP server stopped.


In [10]:
from IPython.core.display import HTML

# Embed an external website using an iframe
iframe_html = """
<iframe src="http://0.0.0.0:8000/analysis/11_caprieval_analysis/report.html" width="90%" height="600" style="border:none;">
    Your browser does not support iframes.
</iframe>
"""
display(HTML(iframe_html))

127.0.0.1 - - [04/Apr/2025 16:22:53] "GET /analysis/11_caprieval_analysis/report.html HTTP/1.1" 200 -
127.0.0.1 - - [04/Apr/2025 16:23:31] "GET /10_seletopclusts/cluster_1_model_4.pdb.gz HTTP/1.1" 200 -
127.0.0.1 - - [04/Apr/2025 16:25:05] "GET /10_seletopclusts/cluster_1_model_2.pdb.gz HTTP/1.1" 200 -


# Barnase-Barstar protein complex

In [ ]:
# Barnase-Barstar protein complex
# From Chen, R., Mintseris, J., Janin, J. and Weng, Z. (2003)
# A protein–protein docking benchmark. 
# Proteins, 52: 88-91. https://doi-org.sire.ub.edu/10.1002/prot.10390
barnase_id = "1A2P"
barnase_ch = "B"
barstar_id = "1A19"
barstar_ch = "A"
complex_id = "1BRS" # barnase_barstar_complex
complex_ch = "A,D"
out_path = '.data/barnase_barstar/'

In [4]:
# Downloading desired PDB files
# Import module
from biobb_io.api.pdb import pdb

# Create properties dict and inputs/outputs
barnase_pdb = f'{out_path}{barnase_id}.pdb'
barstar_pdb = f'{out_path}{barstar_id}.pdb'
complex_pdb = f'{out_path}{complex_id}.pdb'

# Create and launch bb
pdb(output_pdb_path = barnase_pdb, properties = {'pdb_code': barnase_id})
pdb(output_pdb_path = barstar_pdb, properties = {'pdb_code': barstar_id})
pdb(output_pdb_path = complex_pdb, properties = {'pdb_code': complex_id})

os.remove('log.out'), os.remove('log.err')

2025-04-17 09:43:41,652 [MainThread  ] [INFO ]  Module: biobb_io.api.pdb Version: 5.0.1
2025-04-17 09:43:41,653 [MainThread  ] [INFO ]  Downloading 1a2p from: https://www.ebi.ac.uk/pdbe/entry-files/download/pdb1a2p.ent
2025-04-17 09:43:41,950 [MainThread  ] [INFO ]  Writting pdb to: ./barnase_barstar/1A2P.pdb
2025-04-17 09:43:41,951 [MainThread  ] [INFO ]  Filtering lines NOT starting with one of these words: ['ATOM', 'MODEL', 'ENDMDL']
2025-04-17 09:43:41,956 [MainThread  ] [INFO ]  
2025-04-17 09:43:41,959 [MainThread  ] [INFO ]  Module: biobb_io.api.pdb Version: 5.0.1
2025-04-17 09:43:41,959 [MainThread  ] [INFO ]  Downloading 1a19 from: https://www.ebi.ac.uk/pdbe/entry-files/download/pdb1a19.ent
2025-04-17 09:43:42,310 [MainThread  ] [INFO ]  Writting pdb to: ./barnase_barstar/1A19.pdb
2025-04-17 09:43:42,310 [MainThread  ] [INFO ]  Filtering lines NOT starting with one of these words: ['ATOM', 'MODEL', 'ENDMDL']
2025-04-17 09:43:42,312 [MainThread  ] [INFO ]  
2025-04-17 09:43:42,

(None, None)

In [51]:
# These are the pdbs we get from RCSB
show_pdbs([barnase_pdb, barstar_pdb, complex_pdb])

In [ ]:
# Filtering specific chains: we need to get rid of repeated chains
from biobb_pdb_tools.pdb_tools.biobb_pdb_selchain import biobb_pdb_selchain

# Create properties dict and inputs/outputs
barnase_pdb_ch = f'{out_path}{barnase_id}_ch.pdb'
barstar_pdb_ch = f'{out_path}{barstar_id}_ch.pdb'
complex_pdb_ch = f'{out_path}{complex_id}_ch.pdb'

# # Create and launch bb
biobb_pdb_selchain(input_file_path=barnase_pdb,
                   output_file_path=barnase_pdb_ch,
                   properties={'chains': barnase_ch})
os.remove('log.out'), os.remove('log.err')

biobb_pdb_selchain(input_file_path=barstar_pdb,
                   output_file_path=barstar_pdb_ch,
                   properties={'chains': barstar_ch})
os.remove('log.out'), os.remove('log.err')

biobb_pdb_selchain(input_file_path=complex_pdb,
                   output_file_path=complex_pdb_ch,
                   properties={'chains': complex_ch})
os.remove('log.out'), os.remove('log.err')

2025-04-17 09:43:58,647 [MainThread  ] [INFO ]  Module: biobb_pdb_tools.pdb_tools.biobb_pdb_selchain Version: 5.0.0
2025-04-17 09:43:58,648 [MainThread  ] [INFO ]  /home/rchaves/repo/ab_design/biobb_wf/sandbox_35f417df-5305-4215-aad6-6b06aacaff57 directory successfully created
2025-04-17 09:43:58,649 [MainThread  ] [INFO ]  Copy: ./barnase_barstar/1A2P.pdb to /home/rchaves/repo/ab_design/biobb_wf/sandbox_35f417df-5305-4215-aad6-6b06aacaff57
2025-04-17 09:43:58,649 [MainThread  ] [INFO ]  Appending chains to select
2025-04-17 09:43:58,650 [MainThread  ] [INFO ]  pdb_selchain -B /home/rchaves/repo/ab_design/biobb_wf/sandbox_35f417df-5305-4215-aad6-6b06aacaff57/1A2P.pdb > ./barnase_barstar/1A2P_ch.pdb
2025-04-17 09:43:58,650 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2025-04-17 09:43:58,650 [MainThread  ] [INFO ]  pdb_selchain -B /home/rchaves/repo/ab_design/biobb_wf/sandbox_35f417df-5305-4215-aad6-6b06aacaff57/1A2P.pdb > ./barnase_barstar/1A2P_

(None, None)

In [47]:
# On a real case we don't have the reference to know how the proteins bind each other
# What information can use to guide the process?
show_pdbs([barnase_pdb_ch, barstar_pdb_ch, complex_pdb_ch])

In [ ]:
# Solvent accessibility: 
from biobb_haddock.haddock_restraints.haddock3_accessibility import haddock3_accessibility

# Create properties dict and inputs/outputs
barnase_sasa = f'{out_path}{barnase_id}_sasa.txt'
barstar_sasa = f'{out_path}{barstar_id}_sasa.txt'
barnase_actpass = f'{out_path}{barnase_id}_actpass.txt'
barstar_actpass = f'{out_path}{barstar_id}_actpass.txt'

cutoff = 0.3
# Barnase Chain
haddock3_accessibility(
        input_pdb_path            = barnase_pdb_ch,
        output_accessibility_path = barnase_sasa,
        output_actpass_path       = barnase_actpass,
        properties                = {'chain': barnase_ch,
                                     'cutoff': cutoff})
os.remove('log.out'), os.remove('log.err')

# Barstar Chain
haddock3_accessibility(
        input_pdb_path            = barstar_pdb_ch,
        output_accessibility_path = barstar_sasa,
        output_actpass_path       = barstar_actpass,
        properties                = {'chain': barstar_ch,
                                     'cutoff': cutoff})
os.remove('log.out'), os.remove('log.err')


2025-04-17 19:13:02,195 [MainThread  ] [INFO ]  Module: biobb_haddock.haddock_restraints.haddock3_accessibility Version: 5.0.0
2025-04-17 19:13:02,196 [MainThread  ] [INFO ]  /home/rchaves/repo/ab_design/biobb_wf/sandbox_3b936a63-9b2d-4a66-9356-3135a0098624 directory successfully created
2025-04-17 19:13:02,197 [MainThread  ] [INFO ]  Copy: ./barnase_barstar/1A2P_ch.pdb to /home/rchaves/repo/ab_design/biobb_wf/sandbox_3b936a63-9b2d-4a66-9356-3135a0098624
2025-04-17 19:13:02,197 [MainThread  ] [INFO ]  haddock3-restraints calc_accessibility /home/rchaves/repo/ab_design/biobb_wf/sandbox_3b936a63-9b2d-4a66-9356-3135a0098624/1A2P_ch.pdb --export_to_actpass -c 0.3 &> /home/rchaves/repo/ab_design/biobb_wf/sandbox_3b936a63-9b2d-4a66-9356-3135a0098624/1A2P_sasa.txt

2025-04-17 19:13:02,560 [MainThread  ] [INFO ]  Executing: haddock3-restraints calc_accessibility /home/rchaves/repo/ab_design/biobb_wf/san...
2025-04-17 19:13:02,561 [MainThread  ] [INFO ]  Exit code: 0
2025-04-17 19:13:02,563 [Ma

(None, None)

In [53]:
def display_actpass(pdb, actpass):
    with open(actpass, 'r') as file:
        res = file.read()
        res = res.replace(' ', ', ')
        
    # Load the PDB files
    view = nv.NGLWidget()
    view.add_component(pdb)
    view.clear()
    #view.add_surface(selection=res, color='red')
    view.add_surface(selection=f'not ( {res} ) ', color='blue')
    view.add_surface(selection=f'    ( {res} ) ', color='red')
    view.layout.width = '100%'
    return view

# Careful! Pockets are good places to bind but have low accessibility
view1 = display_actpass(barnase_pdb_ch, barnase_actpass)
view2 = display_actpass(barstar_pdb_ch, barstar_actpass)
ipywidgets.HBox([view1, view2])

In [86]:
# Electrostatic energies:
# We see a postive charge in the binding site of barnase and a negative charge in the binding site of barstar
show_pdbs([barnase_pdb_ch, barstar_pdb_ch],surface=True)

In [93]:
v = nv.show_file(barnase_pdb_ch, surface=True)
v.add_ball_and_stick(selection='83')
v

NGLWidget()

In [ ]:
# Choose manually a few active and create passive around them
# Like Arg 83 and Glu 80 from barnase and barstar respectively
# TODO: add passive_from_active
# https://www.bonvinlab.org/haddock3/clients/haddock.clis.restraints.passive_from_active.html
!haddock3-restraints passive_from_active {barnase_pdb_ch} 73
!haddock3-restraints passive_from_active {barstar_pdb_ch} 80

[2025-04-17 19:32:52,598 cli_restraints INFO] [DEPRECATION NOTICE] This command will soon be replaced with `haddock-restraints`
27 35 37 40 54 78 80 81 82 84 85 86 102 103
[2025-04-17 19:32:53,156 cli_restraints INFO] [DEPRECATION NOTICE] This command will soon be replaced with `haddock-restraints`
38 75 76 78 79 81 82


In [ ]:
# visualize the result

In [63]:
# Convert active/passive to ambiguous restraints
from biobb_haddock.haddock_restraints.haddock3_actpass_to_ambig import haddock3_actpass_to_ambig

barnase_barstar_tbl = f'{out_path}barnase_barstar.tbl'

haddock3_actpass_to_ambig( 
    input_actpass1_path=barnase_actpass,
    input_actpass2_path=barstar_actpass,    
    output_tbl_path=barnase_barstar_tbl,
    properties = {'remove_tmp' : True}
)

os.remove('log.out'), os.remove('log.err')

2025-04-17 19:34:27,950 [MainThread  ] [INFO ]  Module: biobb_haddock.haddock_restraints.haddock3_actpass_to_ambig Version: 5.0.0
2025-04-17 19:34:27,951 [MainThread  ] [INFO ]  /home/rchaves/repo/ab_design/biobb_wf/sandbox_27bead45-4dad-4440-89ed-60f3ea2e596d directory successfully created
2025-04-17 19:34:27,952 [MainThread  ] [INFO ]  Copy: ./barnase_barstar/1A2P_actpass.txt to /home/rchaves/repo/ab_design/biobb_wf/sandbox_27bead45-4dad-4440-89ed-60f3ea2e596d
2025-04-17 19:34:27,952 [MainThread  ] [INFO ]  Copy: ./barnase_barstar/1A19_actpass.txt to /home/rchaves/repo/ab_design/biobb_wf/sandbox_27bead45-4dad-4440-89ed-60f3ea2e596d
2025-04-17 19:34:27,952 [MainThread  ] [INFO ]  haddock3-restraints active_passive_to_ambig /home/rchaves/repo/ab_design/biobb_wf/sandbox_27bead45-4dad-4440-89ed-60f3ea2e596d/1A2P_actpass.txt /home/rchaves/repo/ab_design/biobb_wf/sandbox_27bead45-4dad-4440-89ed-60f3ea2e596d/1A19_actpass.txt &> /home/rchaves/repo/ab_design/biobb_wf/sandbox_27bead45-4dad-444

(None, None)

In [64]:
!haddock3-restraints validate_tbl {barnase_barstar_tbl} > {barnase_barstar_tbl}.log

In [65]:
from biobb_haddock.haddock.topology import topology

properties={
    'cfg': {
        'tolerance': 0,
    },
    'remove_tmp' : True,
}
step_idx = 0
barnase_top_zip_path = f'{out_path}{step_idx}/barnase_top.zip'
barstar_top_zip_path = f'{out_path}{step_idx}/barstar_top.zip'
wf_topology          = f'{out_path}{step_idx}/wf_topology.zip'

topology(mol1_input_pdb_path        = barnase_pdb_ch,
         mol2_input_pdb_path        = barstar_pdb_ch,
         mol1_output_top_zip_path   = barnase_top_zip_path,
         mol2_output_top_zip_path   = barstar_top_zip_path,
         output_haddock_wf_data_zip = wf_topology,
         properties                 = properties)

os.remove('log.out'); os.remove('log.err')

2025-04-17 19:34:47,839 [MainThread  ] [INFO ]  Module: biobb_haddock.haddock.topology Version: 5.0.0
2025-04-17 19:34:47,839 [MainThread  ] [INFO ]  /home/rchaves/repo/ab_design/biobb_wf/sandbox_dfb2b9ca-6e36-4c7c-b9c2-cf34ec329ca5 directory successfully created
2025-04-17 19:34:47,840 [MainThread  ] [INFO ]  Copy: ./barnase_barstar/1A2P_ch.pdb to /home/rchaves/repo/ab_design/biobb_wf/sandbox_dfb2b9ca-6e36-4c7c-b9c2-cf34ec329ca5
2025-04-17 19:34:47,840 [MainThread  ] [INFO ]  Copy: ./barnase_barstar/1A19_ch.pdb to /home/rchaves/repo/ab_design/biobb_wf/sandbox_dfb2b9ca-6e36-4c7c-b9c2-cf34ec329ca5
CFG: tolerance
CFG: 0
2025-04-17 19:34:47,841 [MainThread  ] [INFO ]  haddock3 f12072d1-3b45-4adb-a530-3f05000f431e/haddock.cfg

2025-04-17 19:34:49,427 [MainThread  ] [INFO ]  Executing: haddock3 f12072d1-3b45-4adb-a530-3f05000f431e/haddock.cfg...
2025-04-17 19:34:49,428 [MainThread  ] [INFO ]  Exit code: 0
2025-04-17 19:34:49,429 [MainThread  ] [INFO ]  [2025-04-17 19:34:48,503 cli INFO] 
##

In [95]:
from biobb_haddock.haddock.rigid_body import rigid_body

properties={
    'cfg': {
        'tolerance': 5,
        'sampling': 100,
        # turn on random definiton of AIRs
        'ranair': True
    },
}

step_idx = 1
docking_output_zip_path = f'{out_path}{step_idx}/docking.zip'
wf_rigidbody            = f'{out_path}{step_idx}/wf_rigidbody.zip'

rigid_body(input_haddock_wf_data_zip     = wf_topology,
           docking_output_zip_path       = docking_output_zip_path,
        #    ambig_restraints_table_path   = barnase_barstar_tbl,
        #    unambig_restraints_table_path = barnase_barstar_tbl,
        #    hb_restraints_table_path      = barnase_barstar_tbl,
           output_haddock_wf_data_zip    = wf_rigidbody,
           properties                    = properties)

os.remove('log.out'), os.remove('log.err')

2025-04-17 19:54:58,851 [MainThread  ] [INFO ]  Module: biobb_haddock.haddock.rigid_body Version: 5.0.0
2025-04-17 19:54:58,852 [MainThread  ] [INFO ]  /home/rchaves/repo/ab_design/biobb_wf/sandbox_bcd18657-525f-468b-aa6e-c22d56eb214e directory successfully created


2025-04-17 19:54:58,855 [MainThread  ] [INFO ]  Extracting: /home/rchaves/repo/ab_design/biobb_wf/barnase_barstar/0/wf_topology.zip
2025-04-17 19:54:58,855 [MainThread  ] [INFO ]  to:
2025-04-17 19:54:58,856 [MainThread  ] [INFO ]  ['4af773d5-cdf3-444e-9604-b63b842d418d/0_topoaa', '4af773d5-cdf3-444e-9604-b63b842d418d/analysis', '4af773d5-cdf3-444e-9604-b63b842d418d/data', '4af773d5-cdf3-444e-9604-b63b842d418d/log', '4af773d5-cdf3-444e-9604-b63b842d418d/data/0_topoaa', '4af773d5-cdf3-444e-9604-b63b842d418d/data/configurations', '4af773d5-cdf3-444e-9604-b63b842d418d/data/0_topoaa/1A19_ch.pdb', '4af773d5-cdf3-444e-9604-b63b842d418d/data/0_topoaa/1A2P_ch.pdb', '4af773d5-cdf3-444e-9604-b63b842d418d/data/configurations/raw_input.toml', '4af773d5-cdf3-444e-9604-b63b842d418d/data/configurations/cleaned_input.toml', '4af773d5-cdf3-444e-9604-b63b842d418d/data/configurations/README.txt', '4af773d5-cdf3-444e-9604-b63b842d418d/data/configurations/enhanced_haddock_params.json', '4af773d5-cdf3-444e-

(None, None)

In [96]:
# Unzip docking_output_zip_path
with zipfile.ZipFile(docking_output_zip_path, 'r') as zip_ref:
    zip_ref.extractall(docking_output_zip_path[:-4])

In [97]:
import pytraj as pt
import glob

pdb_dir = "barnase_barstar/1/docking/"
pdb_files = sorted(glob.glob(f"{pdb_dir}/*.pdb.gz"))
def show_aligned(chain):
    # Get all PDB files and sort them
    # Create a trajectory from the PDB files
    traj = pt.iterload(pdb_files, top=pdb_files[0])
    # Save the trajectory
    # pt.write_traj(f"{pdb_dir}/combined_{chain}_aligned.dcd", traj, overwrite=True)
    pt.align(traj, ref=0, mask=f'::{chain}')
    traj.save(f"{pdb_dir}/combined_{chain}_aligned_clust.pdb", options="model", overwrite=True)
    view = nv.show_pytraj(traj)
    view.layout.width = '100%'
    return view

In [98]:
view1 = show_aligned('B') # barnase
view2 = show_aligned('A') # barstar

# Display the viewer
ipywidgets.HBox([view1, view2])

In [75]:
view1 = nv.show_structure_file(f"{pdb_dir}/combined_A_aligned_clust.pdb", default_representation=False)
view2 = nv.show_structure_file(f"{pdb_dir}/combined_B_aligned_clust.pdb", default_representation=False)
view1.add_ribbon(color='chainIndex')
view2.add_ribbon(color='chainIndex')
view1.layout.width = '100%'
view2.layout.width = '100%'
# Display the viewer
box = ipywidgets.HBox([view1, view2])
display(box)
# Create a dropdown widget
opts = ['All']
opts.extend([pdb_file.split('/')[-1].split('.')[0] for pdb_file in pdb_files])
mdsel = ipywidgets.Dropdown(
    options=opts,
    description='Sel. model:',
    disabled=False,
)
display(mdsel)

def on_dropdown_change(change):
    """Handle dropdown selection changes.
    From https://github.com/nglviewer/nglview/issues/765
    """
    if change['type'] == 'change' and change['name'] == 'value': 
        selected_file = change['new']
        if selected_file=='All':
            view1._remote_call('setSelection', target='compList', args=["*"], 
               kwargs=dict(component_index=0))
            view2._remote_call('setSelection', target='compList', args=["*"], 
               kwargs=dict(component_index=0))
        else:
            # Extract model number from the filename
            model_num = selected_file.split('_')[1]
            print(f"Selected model: {model_num}")
            # Update the view with the selected model
            view1._remote_call('setSelection', target='compList', 
                            args=[f"/{model_num}"], 
                            kwargs=dict(component_index=0))
            # You can also update view2 if needed
            view2._remote_call('setSelection', target='compList', 
                            args=[f"/{model_num}"], 
                            kwargs=dict(component_index=0))

# Register the callback function
mdsel.observe(on_dropdown_change, names='value')

Dropdown(description='Sel. model:', options=('All', 'rigidbody_1', 'rigidbody_10', 'rigidbody_11', 'rigidbody_…

In [99]:
from biobb_haddock.haddock.capri_eval import capri_eval

step_idx = 2
output_evaluation_zip_path = f'{out_path}{step_idx}/caprieval.zip'
wf_caprieval               = f'{out_path}{step_idx}/wf_caprieval.zip'

capri_eval(input_haddock_wf_data_zip  = wf_rigidbody,
           reference_pdb_path         = complex_pdb_ch,
           output_evaluation_zip_path = output_evaluation_zip_path,
           output_haddock_wf_data_zip = wf_caprieval,
         properties = {'remove_tmp': True})

os.remove('log.out'), os.remove('log.err')

2025-04-17 19:56:21,301 [MainThread  ] [INFO ]  Module: biobb_haddock.haddock.capri_eval Version: 5.0.0
2025-04-17 19:56:21,304 [MainThread  ] [INFO ]  /home/rchaves/repo/ab_design/biobb_wf/sandbox_9b3e653a-362d-4fef-b2e9-1788a0d80192 directory successfully created
2025-04-17 19:56:21,305 [MainThread  ] [INFO ]  Copy: ./barnase_barstar/1BRS_ch.pdb to /home/rchaves/repo/ab_design/biobb_wf/sandbox_9b3e653a-362d-4fef-b2e9-1788a0d80192
2025-04-17 19:56:21,349 [MainThread  ] [INFO ]  Extracting: /home/rchaves/repo/ab_design/biobb_wf/barnase_barstar/1/wf_rigidbody.zip
2025-04-17 19:56:21,349 [MainThread  ] [INFO ]  to:
2025-04-17 19:56:21,350 [MainThread  ] [INFO ]  ['7b637c0b-8746-42b6-b3d3-3f8d8f13074b/0_topoaa', '7b637c0b-8746-42b6-b3d3-3f8d8f13074b/1_rigidbody', '7b637c0b-8746-42b6-b3d3-3f8d8f13074b/analysis', '7b637c0b-8746-42b6-b3d3-3f8d8f13074b/data', '7b637c0b-8746-42b6-b3d3-3f8d8f13074b/traceback', '7b637c0b-8746-42b6-b3d3-3f8d8f13074b/log', '7b637c0b-8746-42b6-b3d3-3f8d8f13074b/tra

(None, None)

In [100]:
with zipfile.ZipFile(wf_caprieval, 'r') as zip:
    zip.extractall(wf_caprieval[:-4])
    
webbrowser.open("http://0.0.0.0:8000/barnase_barstar/2/wf_caprieval/analysis/2_caprieval_analysis/report.html")
!python3 -m http.server

Serving HTTP on 0.0.0.0 port 8000 (http://0.0.0.0:8000/) ...
127.0.0.1 - - [17/Apr/2025 19:56:27] "GET /barnase_barstar/2/wf_caprieval/analysis/2_caprieval_analysis/report.html HTTP/1.1" 200 -

Keyboard interrupt received, exiting.
^C


In [74]:
# Unzip output_evaluation_zip_path
with zipfile.ZipFile(output_evaluation_zip_path, 'r') as out_eval:
    out_eval.extractall(output_evaluation_zip_path[:-4])

In [ ]:
import pandas as pd

tsv_dir = wf_caprieval[:-4]+'/2_caprieval/'
# Load the cluster and single data into pandas DataFrames
cluster_df = pd.read_csv(tsv_dir + 'capri_clt.tsv', sep='\t',comment='#')
single_df = pd.read_csv(tsv_dir + 'capri_ss.tsv', sep='\t',comment='#')

# Display the first few rows of each DataFrame
display(single_df.head())
display(cluster_df.head())

FileNotFoundError: [Errno 2] No such file or directory: './barnase_barstar/2/wf_caprieval/2_caprieval/capri_clt.tsv'

In [71]:
import gzip
import shutil
best_pdb = os.path.normpath(os.path.join(tsv_dir, single_df['model'][0]))
# Decompress the .gz file
with gzip.open(best_pdb + '.gz', 'rb') as f_in:
    with open(best_pdb, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [72]:
show_pdbs([best_pdb, complex_pdb_ch])

In [ ]:
# the reference and the input proteins have diferent number of residues/atoms, so a fit based on rmsd fails

In [ ]:
# TODO: meter en structure utils
from Bio.PDB import PDBParser, PDBIO
from Bio.PDB.cealign import CEAligner

# Parse the structures
parser = PDBParser(QUIET=True)
structure1 = parser.get_structure("complex_pdb_ch", complex_pdb_ch)
structure2 = parser.get_structure("best_pdb", best_pdb)
    
# Perform CE alignment
aligner = CEAligner()
aligner.set_reference(structure1)
aligner.align(structure2)

# Save structure2 to a PDB file
output_pdb_path = "./aligned_structure2.pdb"
io = PDBIO()
io.set_structure(structure2)
io.save(output_pdb_path)

In [68]:
view = nv.show_structure_file(output_pdb_path)
view.add_component(complex_pdb_ch)
view

NGLWidget()

[caprieval]
reference_fname = "/home/rchaves/repo/ab_design/biobb_wf/barnase_barstar/1BRS_ch.pdb"

[seletop]
select = 25

[caprieval]
reference_fname = "/home/rchaves/repo/ab_design/biobb_wf/barnase_barstar/1BRS_ch.pdb"

[flexref]
tolerance = 5

[caprieval]
reference_fname = "/home/rchaves/repo/ab_design/biobb_wf/barnase_barstar/1BRS_ch.pdb"

# ====================================================================



In [93]:
from biobb_haddock.haddock.haddock3_extend import haddock3_extend

haddock_config_path        = f'{out_path}docking-protein-protein-full.cfg'
output_haddock_wf_data_zip = f'{out_path}wf.zip'  

haddock3_extend(input_haddock_wf_data_zip  = wf_caprieval,
                 haddock_config_path        = haddock_config_path,
                 output_haddock_wf_data_zip = output_haddock_wf_data_zip,
                 properties = {'restart_run': 2, 'remove_tmp': True})

os.remove('log.out'), os.remove('log.err')

2025-04-16 17:28:18,274 [MainThread  ] [INFO ]  Module: biobb_haddock.haddock.haddock3_extend Version: 5.0.0
2025-04-16 17:28:18,275 [MainThread  ] [INFO ]  /home/rchaves/repo/ab_design/biobb_wf/sandbox_f6d2c91f-a27b-4841-bf21-3314437b61c9 directory successfully created
2025-04-16 17:28:18,277 [MainThread  ] [INFO ]  Copy: ./barnase_barstar/2/wf_caprieval.zip to /home/rchaves/repo/ab_design/biobb_wf/sandbox_f6d2c91f-a27b-4841-bf21-3314437b61c9
2025-04-16 17:28:18,277 [MainThread  ] [INFO ]  Copy: ./barnase_barstar/docking-protein-protein-full.cfg to /home/rchaves/repo/ab_design/biobb_wf/sandbox_f6d2c91f-a27b-4841-bf21-3314437b61c9
2025-04-16 17:28:18,296 [MainThread  ] [INFO ]  haddock3 /home/rchaves/repo/ab_design/biobb_wf/sandbox_f6d2c91f-a27b-4841-bf21-3314437b61c9/docking-protein-protein-full.cfg --extend-run 8c727563-104f-4678-9e63-fc8def362532



/home/rchaves/miniforge3/envs/biobb_haddock/lib/python3.9/site-packages/biobb_common/generic/biobb_object.py:183: UserWarning: Warning: restart_run is not a recognized property. The most similar property is: restart
  warnings.warn(


2025-04-16 17:29:33,682 [MainThread  ] [INFO ]  Executing: haddock3 /home/rchaves/repo/ab_design/biobb_wf/sandbox_f6d2c91f-a27b-4841-bf21-3...
2025-04-16 17:29:33,683 [MainThread  ] [INFO ]  Exit code: 0
2025-04-16 17:29:33,683 [MainThread  ] [INFO ]  [2025-04-16 17:28:18,880 cli INFO] 
##############################################
#                                            #
#                 HADDOCK3                   #
#                                            #
##############################################

!! Some of the HADDOCK3 components use CNS (Crystallographic and NMR System) which is free of use for non-profit applications. !!
!! For commercial use it is your own responsibility to have a proper license. !!
!! For details refer to the DISCLAIMER file in the HADDOCK3 repository. !!

Starting HADDOCK3 v2024.10.0b7 on 2025-04-16 17:28:00

Python 3.9.21 | packaged by conda-forge | (main, Dec  5 2024, 13:51:40) 
[GCC 13.3.0]

[2025-04-16 17:28:18,891 prepare_run INFO] Unco

(None, None)